In [11]:
import sys
sys.path.append("./base_model.py")
from base_model import BaseModel

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
import torchvision
import torch.optim as optim
import torch
import torch.nn as nn
import torchvision.transforms as transforms

In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [15]:
transformer =transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the images to (224, 224)
    transforms.ToTensor()])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transformer )
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True,transform=transformer)
testloader = torch.utils.data.DataLoader(testset, batch_size=128,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [16]:
AlexNet = BaseModel()
alexnet = nn.Sequential(*list(AlexNet.model.children())[:-1])
alexnet.to(device)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (1): AdaptiveAvgPool2d(output_size=(6, 6))
)

In [18]:
features = []
labels = []
with torch.no_grad():
    for i,data in enumerate(trainloader, 0):
        images, target = data
        images, target = images.to(device), target.to(device)
        feats = alexnet(images)
        feats = feats.view(feats.size(0), -1)
        features.append(feats)
        labels.append(target)
features = torch.cat(features)
labels = torch.cat(labels)

In [19]:
classifier = nn.Linear(features.size(1), 10)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(classifier.parameters(), lr=0.001, momentum=0.9)


classifier.to(device)

# Train the classifier
classifier.train()
for epoch in range(10):
    optimizer.zero_grad()
    outputs = classifier(features)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

# Evaluate the classifier on test data
classifier.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        feats = alexnet(images)
        feats = feats.view(feats.size(0), -1)
        outputs = classifier(feats)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

accuracy_feature_extraction = 100 * correct / total
print('Test accuracy after feature extraction: {:.2f}%'.format(accuracy_feature_extraction))
torch.save(classifier, "./non_fine_tune.pt")


Test accuracy after feature extraction: 35.24%
